In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.utils import resample
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import add_dummy_feature
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

import scikitplot as skplt
import mord

from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve

from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

### Load and Explore Data

In [3]:
df = pd.read_csv('winequality-red.csv', delimiter=';')

In [4]:
# Relabel
# df.quality = pd.cut(df.quality, bins=3, labels=[1, 2, 3]).astype('int')
df.quality = df.quality.map(lambda x: x-3)

In [5]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2


### Subset Data

In [6]:
df_subset = df.copy()
df_subset['quality2'] = pd.cut(df.quality, bins=3, labels=[1, 2, 3]).astype('int') #['good', 'medium', 'bad'])

### Split Data 

In [7]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [8]:
# Split 
# y = resampled_df.quality
# X = resampled_df.drop(columns='quality')

# y = df_subset.quality2
# X = df_subset.drop(columns=['quality', 'quality2'])

y = df.quality
X = df[['fixed acidity', 'volatile acidity', 'free sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

# y = df.quality
# X = df.drop(columns='quality')

# X = df[['volatile acidity', 'citric acid', 'total sulfur dioxide', 'density', 'alcohol']]
# X = df[['volatile acidity', 'alcohol']]

# # Make sure intercept exists
# if ~any(X.columns == 'Intercept'):
#     X.insert(0, 'Intercept', 1)
# else: 
#     X.Intercept = 1
    
# Stratified Split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, stratify=y, test_size=.7, random_state=129)

In [9]:
def normalize_confusion_matrix(cm):
    
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm = np.around(cm, decimals=2)
    cm[np.isnan(cm)] = 0.0
    
    cm = (cm - np.min(cm)) / (np.max(cm) - np.min(cm))
    
    return cm

In [10]:
def discrete_heatmap(cm):
    
    ### Discrete Confusion Matrix Heatmap
    from sklearn.preprocessing import normalize
    import matplotlib.pyplot as plt
    import seaborn as sns

    fig, ax = plt.subplots(figsize=(10, 6))

    #     # Normalize across predictions
    #     norm_Z = normalize(cm, axis=1)
    #     # Normalize across predictions and truth [Note: Use this if classes are balanced]
    #     norm_Z = (Z - np.min(Z)) / (np.max(Z) - np.min(Z))

    #     norm_Z = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    #     norm_Z = np.around(norm_Z, decimals=2)
    #     norm_Z[np.isnan(norm_Z)] = 0.0
    
    Z = normalize_confusion_matrix(cm)

    # Generate heatmap
    sns.heatmap(Z, cmap='RdYlGn', annot=True, cbar=True)

    ## Format axes object
    #  Format X-axis label and ticks
    ax.set_xlabel('Predicted', fontdict={'size': 12})
    ax.xaxis.set_ticks_position('top') 
    ax.xaxis.set_label_position('top')
    # Format Y-axis label
    ax.set_ylabel('Truth', fontdict={'size': 12})
    # Format Title [Note: double newlines are to create space between Title and the X-axis label that was moved to the top]
    ax.set_title('Red Wine Quality Confusion Matrix\n\n', fontdict={'size': 14, 'weight': 'bold'})

In [11]:
def interpolated_heatmap(cm):
    
    ### Interpolated Confusion Matrix Heatmap
    import numpy as np
    from sklearn.preprocessing import normalize
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 6))

    #     # Normalize across predictions [Note: Use this if classes are imbalanced]
    #     norm_Z = normalize(cm, axis=1)
    #     # Normalize across predictions and truth [Note: Use this if classes are balanced]
    #     norm_Z = (Z - np.min(Z)) / (np.max(Z) - np.min(Z))

    #     norm_Z = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    #     norm_Z = np.around(norm_Z, decimals=2)
    #     norm_Z[np.isnan(norm_Z)] = 0.0
    
    Z = normalize_confusion_matrix(cm)

    # Generate X, Y indices of confusion matrix
    n_classes = Z.shape[0]
    X, Y = np.meshgrid(np.arange(0, n_classes, 1), np.arange(n_classes-1, -1, -1))

    # Plot contours
    plt.contourf(X, Y, Z, 25, cmap='RdYlGn')#, vmin=0, vmax=1)
    ax2 = plt.colorbar();

    ## Format X-axis label and ticks
    ax.set_xlabel('Predicted', fontdict={'size': 12})
    # Move ticks and label to top
    ax.xaxis.set_ticks_position('top') 
    ax.xaxis.set_label_position('top')
    # Hide major tick labels
    ax.set_xticklabels([], minor=False) 
    # Create minor ticks in between and label every other one
    ax.set_xticks([0.5, 1.5, 2.5, 3.5, 4.5], minor=True)
    ax.set_xticklabels(['Low', '', 'Med', '', 'High'], minor=True)

    ## Format Y-axis label and ticks
    ax.set_ylabel('Truth', fontdict={'size': 12})
    # Hide major tick labels
    ax.set_yticklabels([], minor=False)
    # Create minor ticks in between and label every other one
    ax.set_yticks([0.5, 1.5, 2.5, 3.5, 4.5], minor=True)
    ax.set_yticklabels(['High', '', 'Med', '', 'Low'], minor=True)

    # Format Title [Note: double newlines are to create space between Title and the X-axis label that was moved to the top]
    ax.set_title('Red Wine Quality\n\n', fontdict={'size': 14, 'weight': 'bold'})

In [12]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2


In [13]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,2.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,2.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,3.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,3.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,5.000000


### Build Models

In [14]:
def error_distance_cost(y_true, y_pred):
    
    cm = confusion_matrix(y_true, y_pred)
    cm = normalize_confusion_matrix(cm)
    
    cost = np.zeros_like(cm)
    rows, cols = cm.shape
    for ridx in range(rows):
        for cidx in range(cols):
            cost[ridx, cidx] = np.abs(ridx-cidx) * cm[ridx, cidx]

    return np.around(1/np.sum(cost), decimals=5)

def f1(y_true, y_pred):
    
    return f1_score(y_true, y_pred, average='macro')

In [15]:
lr0 = LogisticRegression(class_weight='balanced', 
                         random_state=129,
                         max_iter=10000)
scoring = {'Accuracy': 'accuracy',
#            'F1': make_scorer(f1), 
           'ErrorDistance': make_scorer(error_distance_cost)}
param_grid = [{'C': np.logspace(-9, 9, 19), 
               'penalty': ['l1', 'l2'],
               'multi_class': ['ovr'],
               'solver': ['liblinear', 'saga']}]

searchcv = GridSearchCV(lr0, param_grid, scoring=scoring, cv=3, 
                        return_train_score=True, refit='ErrorDistance')

searchcv.fit(X_train, y_train)

/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: T

/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: T

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=10000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=129,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': array([  1.00000e-09,   1.00000e-08,   1.00000e-07,   1.00000e-06,
         1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05,   1.00000e+06,
         1.00000e+07,   1.00000e+08,   1.00000e+09]), 'penalty': ['l1', 'l2'], 'multi_class': ['ovr'], 'solver': ['liblinear', 'saga']}],
       pre_dispatch='2*n_jobs', refit='ErrorDistance',
       return_train_score=True,
       scoring={'Accuracy': 'accuracy', 'ErrorDistance': make_scorer(error_distance_cost)},
       verbose=0

In [16]:
print(searchcv.best_score_)
print(1 / searchcv.best_score_)

0.209823674322
4.7659064366


In [17]:
searchcv.best_estimator_

LogisticRegression(C=10.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=10000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=129,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [18]:
searchcv.cv_results_

{'mean_fit_time': array([  1.79449717e-03,   4.16572889e-03,   1.66885058e-03,
          9.21861331e-03,   1.20949745e-03,   3.46509616e-03,
          1.62450473e-03,   6.72086080e-03,   1.13407771e-03,
          3.52136294e-03,   1.67576472e-03,   7.07451502e-03,
          1.20854378e-03,   3.32776705e-03,   1.71438853e-03,
          1.10461712e-02,   1.26886368e-03,   3.64327431e-03,
          2.04435984e-03,   6.39929771e-02,   1.27919515e-03,
          3.58168284e-03,   2.56045659e-03,   9.59526698e-01,
          1.67767207e-03,   8.24069977e-03,   3.05612882e-03,
          1.46377500e+00,   3.51587931e-03,   1.76154558e+00,
          3.05668513e-03,   2.45310966e+00,   9.89429156e-03,
          2.84012071e+00,   4.57024574e-03,   1.98485843e+00,
          8.62792333e-02,   1.75921623e+00,   5.35416603e-03,
          1.98071210e+00,   3.98034016e-01,   2.48949742e+00,
          6.60061836e-03,   1.97581196e+00,   1.60429136e+00,
          2.79287799e+00,   7.83069928e-03,   2.29438

In [19]:
searchcv.cv_results_['mean_test_F1'][searchcv.cv_results_['mean_test_ErrorDistance'] == searchcv.best_score_]

KeyError: 'mean_test_F1'

In [ ]:
searchcv.cv_results_

In [ ]:
df.head()

In [ ]:
df_result = pd.concat([pd.Series(lr2_pred, name='Class'), pd.DataFrame(lr2_probas)], 
                      axis=1, names=[str(c) for c in range(lr2_probas.shape[1])])
df_result

In [ ]:
lr2.coef_.shape

In [ ]:
lr2.coef_.shape

In [ ]:
lr2.intercept_

In [ ]:
lr1 = LogisticRegression(C=1e3, 
                         class_weight='balanced', 
                         random_state=129,
                         multi_class='ovr',
                         solver='liblinear')
pipe1 = make_pipeline(StandardScaler(), lr2)

pipe1.fit(X_train, y_train)

print(pipe1)

# make predictions
lr1_pred = pipe1.predict(X_test)
lr1_probas = pipe1.predict_proba(X_test)

# summarize the fit of the model
print(classification_report(y_test, lr1_pred))
print(confusion_matrix(y_test, lr1_pred))

In [ ]:
lr2 = LogisticRegression(C=1e3, 
                         class_weight='balanced', 
                         random_state=129,
                         multi_class='ovr',
                         solver='liblinear')
lr2.fit(X_train, y_train)
# lr2.fit(X_train, y_train).decision_function(X_test, )

print(lr2)

# make predictions
lr2_pred = lr2.predict(X_test)
lr2_probas = lr2.predict_proba(X_test)

# summarize the fit of the model
print(classification_report(y_test, lr2_pred))
print(confusion_matrix(y_test, lr2_pred))

In [ ]:
adjusted_rand_score(y_test, lr2_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_confusion_matrix(y_test, lr2_pred, 
                                    normalize=True, cmap='RdYlGn', ax=ax)

discrete_heatmap(confusion_matrix(y_test, lr2_pred))

interpolated_heatmap(confusion_matrix(y_test, lr2_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_roc_curve(y_test, lr2_probas, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_precision_recall_curve(y_test, lr2_probas, ax=ax)

In [ ]:
lr3.coef_.shape

In [ ]:
lr3 = LogisticRegression(C=1e3, 
                         class_weight='balanced', 
                         random_state=129,
                         multi_class='multinomial',
                         solver='newton-cg', max_iter=1000)
lr3.fit(X_train, y_train)

print(lr3)

# make predictions
lr3_pred = lr3.predict(X_test)
lr3_probas = lr3.predict_log_proba(X_test)

# summarize the fit of the model
print(classification_report(y_test, lr3_pred))
print(confusion_matrix(y_test, lr3_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_confusion_matrix(y_test, lr3_pred, 
                                    normalize=True, cmap='RdYlGn', ax=ax)

discrete_heatmap(confusion_matrix(y_test, lr3_pred))

interpolated_heatmap(confusion_matrix(y_test, lr3_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_roc_curve(y_test, lr3_probas, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_precision_recall_curve(y_test, lr3_probas, ax=ax)

In [ ]:
lr4 = LogisticRegression(C=1e3, 
                         class_weight='balanced', 
                         random_state=129,
                         multi_class='multinomial',
                         solver='lbfgs')
lr4.fit(X_train, y_train)

print(lr4)

# make predictions
lr4_pred = lr4.predict(X_test)
lr4_probas = lr4.predict_log_proba(X_test)

# summarize the fit of the model
print(classification_report(y_test, lr4_pred))
print(confusion_matrix(y_test, lr4_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_confusion_matrix(y_test, lr4_pred, 
                                    normalize=True, cmap='RdYlGn', ax=ax)

discrete_heatmap(confusion_matrix(y_test, lr4_pred))

interpolated_heatmap(confusion_matrix(y_test, lr4_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_roc_curve(y_test, lr4_probas, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_precision_recall_curve(y_test, lr4_probas, ax=ax)

In [ ]:
lr5 = LogisticRegression(C=1e3, 
                         class_weight='balanced', 
                         random_state=129,
                         multi_class='multinomial',
                         solver='sag',
                         max_iter=10000*2.5)
lr5.fit(X_train, y_train)

print(lr5)

# make predictions
lr5_pred = lr5.predict(X_test)
lr5_probas = lr5.predict_log_proba(X_test)

# summarize the fit of the model
print(classification_report(y_test, lr5_pred))
print(confusion_matrix(y_test, lr5_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_confusion_matrix(y_test, lr5_pred, 
                                    normalize=True, cmap='RdYlGn', ax=ax)

discrete_heatmap(confusion_matrix(y_test, lr5_pred))

interpolated_heatmap(confusion_matrix(y_test, lr5_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_roc_curve(y_test, lr5_probas, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_precision_recall_curve(y_test, lr5_probas, ax=ax)

In [ ]:
lr6 = LogisticRegression(C=1e3, 
                         class_weight='balanced', 
                         random_state=129,
                         multi_class='multinomial',
                         solver='saga',
                         max_iter=10000*2.5)
lr6.fit(X_train, y_train)

print(lr6)

# make predictions
lr6_pred = lr6.predict(X_test)
lr6_probas = lr6.predict_log_proba(X_test)

# summarize the fit of the model
print(classification_report(y_test, lr6_pred))
print(confusion_matrix(y_test, lr6_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_confusion_matrix(y_test, lr6_pred, 
                                    normalize=True, cmap='RdYlGn', ax=ax)

discrete_heatmap(confusion_matrix(y_test, lr6_pred))

interpolated_heatmap(confusion_matrix(y_test, lr6_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_roc_curve(y_test, lr6_probas, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_precision_recall_curve(y_test, lr6_probas, ax=ax)

### Plot ROC Curves

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# Binarize the output
classes = sorted(y.unique())
print(classes)
# y2 = label_binarize(y, classes=classes)
lb = LabelBinarizer()
y2 = lb.fit_transform(y)
n_classes = y2.shape[1]
# Resplit using same seed
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=.7, random_state=129)

In [ ]:
# Learn to predict each class against the other
classifier = OneVsRestClassifier(LogisticRegression(C=1000, class_weight='balanced', random_state=129))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)
y_pred = classifier.predict(X_test)
y_probas = classifier.predict_proba(X_test)

print(classifier)
# print(classification_report(y_test, y_pred))
print(classification_report(lb.inverse_transform(y_test), lb.inverse_transform(y_pred)))
print(confusion_matrix(lb.inverse_transform(y_test), lb.inverse_transform(y_pred)))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_confusion_matrix(lb.inverse_transform(y_test), 
                                    lb.inverse_transform(y_pred), 
                                    normalize=True, cmap='RdYlGn', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_roc_curve(lb.inverse_transform(y_test), y_probas, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
skplt.metrics.plot_precision_recall_curve(lb.inverse_transform(y_test), y_probas, ax=ax)

In [ ]:
skplt.metrics.plot_calibration_curve([lb.inverse_transform(y_test)], [y_probas])